# 2B

In [997]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel, SelectKBest
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
import sklearn.preprocessing as preprocessing

In [998]:
random_state = 42

In [999]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas._libs.parsers import k
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV

In [1000]:
mvp_stats_2b = pd.read_csv("datasets/player_mvp_stats.csv", ';')
nicknames_df = pd.read_csv("datasets/nicknames.csv", ';')
salaries_2b = pd.read_csv("datasets/salaries.csv")
teams_2b = pd.read_csv("datasets/teams.csv", ';')
us_inflation_2b = pd.read_csv("datasets/US CPI.csv")

C:\Users\imman\AppData\Local\Temp\ipykernel_25880\3205523321.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  mvp_stats_2b = pd.read_csv("datasets/player_mvp_stats.csv", ';')
C:\Users\imman\AppData\Local\Temp\ipykernel_25880\3205523321.py:2: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  nicknames_df = pd.read_csv("datasets/nicknames.csv", ';')
C:\Users\imman\AppData\Local\Temp\ipykernel_25880\3205523321.py:4: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  teams_2b = pd.read_csv("datasets/teams.csv", ';')


### 2B. Preprocess dan merge beberapa dataset menjadi satu

In [1001]:
def dataframe_to_dict(nicknames_df, key_column_name='Name', value_column_name='Abbreviation'):
    keys = nicknames_df[key_column_name]
    values = nicknames_df[value_column_name]
    return dict(zip(keys, values))

In [1002]:
def cleaning_teamsdf_removeUnimportantRows(teams_2b):
    teams_2b = teams_2b.copy()
    important_rows = ~teams_2b.W.str.contains("Division")
    return teams_2b[important_rows]

def cleaning_teamsdf_convertStripToNan(teams_2b):
    teams_2b = teams_2b.copy()
    teams_2b.GB = teams_2b.GB.replace("—", np.NAN)
    return teams_2b

def cleaning_teamsdf_convertToAppropriateDatatype(teams_2b):
    """
    preconditions:
        cleaning_teamsdf_removeUnimportantRows
         cleaning_teamsdf_convertStripToNan
    """
    teams_2b = teams_2b.copy()
    teams_2b.W = pd.to_numeric(teams_2b.W)
    teams_2b.L = pd.to_numeric(teams_2b.L)
    teams_2b['W/L%'] = pd.to_numeric(teams_2b['W/L%'])
    teams_2b.GB = pd.to_numeric(teams_2b.GB)
    teams_2b['PS/G'] = pd.to_numeric(teams_2b['PS/G'])
    teams_2b['PA/G'] = pd.to_numeric(teams_2b['PA/G'])
    teams_2b.SRS = pd.to_numeric(teams_2b.SRS)
    return teams_2b

def cleaning_anyDf_removeStarInTeamName(teams_2b, column_name='Team'):
    teams_2b = teams_2b.copy()
    teams_2b[column_name] = teams_2b[column_name].str.replace('*', '')
    return teams_2b


def cleaning_anyDf_convertTeamNamesToItsAbbrev(teams_2b, nicknames_df, column_name='Team'):
    """
    precondition:
        teams df:
            cleaning_anyDf_removeStarInTeamName
    """
    nicknames_df = nicknames_df.copy()
    teams_2b = teams_2b.copy()

    nicknames_df = nicknames_df.drop_duplicates(subset='Name', keep='first')
    mapping = dataframe_to_dict(nicknames_df)
    mapping |= dataframe_to_dict(nicknames_df, 'Abbreviation', 'Abbreviation')

    teams_2b[column_name] = list(map(
        lambda x: mapping[x],
        teams_2b[column_name]
    ))
    return teams_2b

In [1003]:
teams_2b = cleaning_teamsdf_removeUnimportantRows(teams_2b)
teams_2b = cleaning_teamsdf_convertStripToNan(teams_2b)
teams_2b = cleaning_teamsdf_convertToAppropriateDatatype(teams_2b)
teams_2b = cleaning_anyDf_removeStarInTeamName(teams_2b)
teams_2b = cleaning_anyDf_convertTeamNamesToItsAbbrev(teams_2b, nicknames_df)
teams_2b

C:\Users\imman\AppData\Local\Temp\ipykernel_25880\1280597476.py:29: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  teams_2b[column_name] = teams_2b[column_name].str.replace('*', '')


,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,683.0,NaN,111.5,105.7,5.22,1991,BOS
1,44,38,537.0,12.0,105.4,105.6,-0.39,1991,PHI
2,39,43,476.0,17.0,103.1,103.3,-0.43,1991,NYK
3,30,52,366.0,26.0,101.4,106.4,-4.84,1991,WSB
4,26,56,317.0,30.0,102.9,107.5,-4.53,1991,NJN
...,...,...,...,...,...,...,...,...,...
1028,42,30,583.0,NaN,112.4,110.2,2.26,2021,DAL
1029,38,34,528.0,4.0,113.3,112.3,1.07,2021,MEM
1030,33,39,458.0,9.0,111.1,112.8,-1.58,2021,SAS
1031,31,41,431.0,11.0,114.6,114.9,-0.20,2021,NOP


In [1004]:
from datetime import datetime


def cleaning_inflationDf_separateDateColumns(us_inflation_2b, column='Yearmon', format="%d-%m-%Y"):
    us_inflation_2b = us_inflation_2b.copy()

    dates = []
    for i in range(len(us_inflation_2b)):
        date = us_inflation_2b[column].iloc[i]
        date = datetime.strptime(date, format)
        dates.append(date)
    us_inflation_2b['Year']  = [date.year for date in dates]
    us_inflation_2b['Month'] = [date.month for date in dates]
    us_inflation_2b['Day']   = [date.day for date in dates]

    return us_inflation_2b


def cleaning_inflationDf_getYearByMedian(us_inflation_2b):
    """
    precondition: cleaning_inflationDf_separateDateColumns
    """
    us_inflation_2b = us_inflation_2b.copy()
    us_inflation_2b = cleaning_inflationDf_separateDateColumns(us_inflation_2b)
    us_inflation_2b = us_inflation_2b.groupby(by='Year')['CPI'].median()
    us_inflation_2b = us_inflation_2b.reset_index()
    return us_inflation_2b


def cleaning_anyDf_removeDuplicatedColumns(combined_df):
    combined_df = combined_df.copy()
    columns = combined_df.columns

    columns_tobe_removed = []
    for i in range(len(columns)):
        col1 = columns[i]
        for j in range(i+1, len(columns)):
            col2 = columns[j]

            if (combined_df[col1] == combined_df[col2]).all():
                columns_tobe_removed.append(col1)
    columns_tobe_kept = set(combined_df.columns) - set(columns_tobe_removed)
    columns_tobe_kept = list(columns_tobe_kept)
    return combined_df.loc[:, columns_tobe_kept]

def cleaning_anyDf_recalculateWLpercentage(teams_2b):
    teams_2b = teams_2b.copy()
    teams_2b['W/L%'] = 100*teams_2b.W / (teams_2b.W + teams_2b.L)
    return teams_2b

In [1005]:
us_inflation_2b_cleaned = us_inflation_2b.copy()
us_inflation_2b_cleaned = cleaning_inflationDf_separateDateColumns(us_inflation_2b_cleaned)
us_inflation_2b_cleaned = cleaning_inflationDf_getYearByMedian(us_inflation_2b_cleaned)
combined_df = us_inflation_2b_cleaned.merge(teams_2b, left_on=["Year"], right_on=["Year"])
combined_df = cleaning_anyDf_recalculateWLpercentage(combined_df)
combined_df.head(3)

,Year,CPI,W,L,W/L%,GB,PS/G,PA/G,SRS,Team
0,1991,136.1,56,26,68.292683,NaN,111.5,105.7,5.22,BOS
1,1991,136.1,44,38,53.658537,12.0,105.4,105.6,-0.39,PHI
2,1991,136.1,39,43,47.560976,17.0,103.1,103.3,-0.43,NYK


In [1006]:
mvp_stats_2b = mvp_stats_2b.drop('Tm', axis=1)  # karena sudah ada kolom Team
mvp_stats_2b = cleaning_anyDf_convertTeamNamesToItsAbbrev(mvp_stats_2b, nicknames_df)

In [1007]:
display(mvp_stats_2b.head(2))
display(combined_df.head(2))
print('banyaknya null salaries: ', salaries_2b.Salaries.isna().sum())
print('banyaknya null player name salaries: ', salaries_2b.Name.isna().sum())
print('banyaknya null player name mvp stats: ', mvp_stats_2b.Player.isna().sum())
print('banyaknya null team name: ', combined_df.Team.isna().sum())
print()
print('banyaknya row salaries df : ', len(salaries_2b))
print('banyaknya row mvp statsdf : ', len(mvp_stats_2b))
print('banyaknya row combined df : ', len(combined_df))

,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Pts Won,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,A.C. Green,PF,27,82,21,26.4,3.1,6.6,476.0,0.1,0.7,0.2,3.0,5.9,507.0,486.00,2.7,3.7,738.0,2.5,3.8,6.3,0.9,0.7,0.3,1.2,1.4,9.1,1991,0,0,0.0,LAL,58,24,707.0,5.0,106.3,99.6,6.73
1,Byron Scott,SG,29,82,82,32.1,6.1,12.8,477.0,0.9,2.7,324.0,5.2,10.1,517.0,0.51,1.4,1.8,797.0,0.7,2.3,3.0,2.2,1.2,0.3,1.0,1.8,14.5,1991,0,0,0.0,LAL,58,24,707.0,5.0,106.3,99.6,6.73


,Year,CPI,W,L,W/L%,GB,PS/G,PA/G,SRS,Team
0,1991,136.1,56,26,68.292683,NaN,111.5,105.7,5.22,BOS
1,1991,136.1,44,38,53.658537,12.0,105.4,105.6,-0.39,PHI


banyaknya null salaries:  28074
banyaknya null player name salaries:  0
banyaknya null player name mvp stats:  0
banyaknya null team name:  0

banyaknya row salaries df :  37420
banyaknya row mvp statsdf :  14092
banyaknya row combined df :  906


In [1008]:
salaries_2b = salaries_2b[["Name", "Year", "Salaries", "Rank"]]

combined_df = mvp_stats_2b.merge(combined_df, left_on=["Team", "Year"], right_on=["Team", "Year"])
combined_df = salaries_2b.merge(combined_df, left_on=["Name", "Year"], right_on=["Player", "Year"])
combined_df = cleaning_anyDf_removeDuplicatedColumns(combined_df)

combined_df.head(5)

,PS/G_y,ORB,G,Age,FG%,W_y,SRS_y,GB_y,3P%,3P,eFG%,Player,Pts Max,Salaries,Year,FG,L_y,STL,2P,BLK,Pos,FT,TRB,Pts Won,Rank,GS,W/L%_y,2PA,PTS,AST,CPI,DRB,GB_x,Share,PA/G_y,FTA,TOV,MP,FT%,2P%,FGA,Team,3PA,W/L%_x,PF
0,100.8,4.3,79,27,574.0,67,8.41,NaN,0.00,0.0,574.0,Shaquille O'Neal,1210,17142000.0,2000,12.1,15,0.5,12.1,3.0,C,5.5,13.6,1207,1,79,81.707317,21.1,29.7,3.8,172.6,9.4,0.0,998.0,92.3,10.4,2.8,40.0,524.0,575.0,21.1,LAL,0.0,817.00,3.2
1,98.5,2.8,81,23,497.0,50,2.67,5.0,0.37,0.4,507.0,Kevin Garnett,1210,16806000.0,2000,9.4,32,1.5,9.0,1.6,PF,3.8,11.8,408,2,81,60.975610,17.8,22.9,5.0,172.6,9.0,5.0,337.0,96.0,5.0,3.3,40.0,765.0,504.0,18.8,MIN,1.0,0.61,2.5
2,94.4,2.7,79,29,551.0,52,2.75,NaN,0.00,0.0,551.0,Alonzo Mourning,1210,15004000.0,2000,8.3,30,0.5,8.3,3.7,C,5.2,9.5,367,3,78,63.414634,14.9,21.7,1.6,172.6,6.8,0.0,303.0,91.3,7.4,2.7,34.8,711.0,553.0,15.0,MIA,0.1,634.00,3.9
3,96.6,1.6,82,26,459.0,29,-3.47,23.0,0.00,0.0,459.0,Juwan Howard,0,15000000.0,2000,6.2,53,0.8,6.2,0.3,PF,2.5,5.7,0,4,82,35.365854,13.4,14.9,3.0,172.6,4.1,23.0,0.0,99.9,3.4,2.7,35.5,735.0,462.0,13.5,WAS,0.1,354.00,3.6
4,97.5,1.4,82,34,451.0,59,6.36,8.0,327.00,1.0,501.0,Scottie Pippen,0,14795000.0,2000,4.7,23,1.4,3.7,0.5,SF,2.0,6.3,0,5,82,71.951220,7.3,12.5,5.0,172.6,4.9,8.0,0.0,91.0,2.7,2.5,33.5,717.0,506.0,10.5,POR,3.2,0.72,2.5


### 2B. Drop missing values pada target feature

kami memutuskan untuk mendrop missing values pada target feature karena target feature merupakan ground truth. Oleh karena itu, kami tidak setuju apabila kita melakukan imputasi untuk missing values pada kolom ini. Sebab, apabila kita melakukan imputasi terhadap missing values untuk kolom ini, maka kolom ini tidak sepenuhnya menjadi ground truth lagi karena diisikan dengan data-data sintesis yang tidak diketahui kebenarannya.

In [1009]:
print("Banyak missing values pada kolom Salaries: ", combined_df.Salaries.isna().sum())
combined_df = combined_df[combined_df.Salaries.notna()]
combined_df.isna().sum()

Banyak missing values pada kolom Salaries:  390


PS/G_y         0
ORB            0
G              0
Age            0
FG%           25
W_y            0
SRS_y          0
GB_y        1571
3P%         1117
3P             0
eFG%          25
Player         0
Pts Max        0
Salaries       0
Year           0
FG             0
L_y            0
STL            0
2P             0
BLK            0
Pos            0
FT             0
TRB            0
Pts Won        0
Rank           0
GS             0
W/L%_y         0
2PA            0
PTS            0
AST            0
CPI            0
DRB            0
GB_x           0
Share          0
PA/G_y         0
FTA            0
TOV            0
MP             0
FT%          230
2P%           43
FGA            0
Team           0
3PA            0
W/L%_x         0
PF             0
dtype: int64

### 2B. Membagi dataset menjadi training, testing, dan final testing

In [1010]:
x_2b, y_2b = combined_df.drop("Salaries", axis=1), combined_df[["Salaries"]]
x_2b.drop("Team", axis=1, inplace=True)
x_2b.drop("Player", axis=1, inplace=True)
x_2b.drop("Rank", axis=1, inplace=True)

display(x_2b.head(2))
display(y_2b.head(2))

,PS/G_y,ORB,G,Age,FG%,W_y,SRS_y,GB_y,3P%,3P,eFG%,Pts Max,Year,FG,L_y,STL,2P,BLK,Pos,FT,TRB,Pts Won,GS,W/L%_y,2PA,PTS,AST,CPI,DRB,GB_x,Share,PA/G_y,FTA,TOV,MP,FT%,2P%,FGA,3PA,W/L%_x,PF
0,100.8,4.3,79,27,574.0,67,8.41,NaN,0.00,0.0,574.0,1210,2000,12.1,15,0.5,12.1,3.0,C,5.5,13.6,1207,79,81.707317,21.1,29.7,3.8,172.6,9.4,0.0,998.0,92.3,10.4,2.8,40.0,524.0,575.0,21.1,0.0,817.00,3.2
1,98.5,2.8,81,23,497.0,50,2.67,5.0,0.37,0.4,507.0,1210,2000,9.4,32,1.5,9.0,1.6,PF,3.8,11.8,408,81,60.975610,17.8,22.9,5.0,172.6,9.0,5.0,337.0,96.0,5.0,3.3,40.0,765.0,504.0,18.8,1.0,0.61,2.5


,Salaries
0,17142000.0
1,16806000.0


### 2B training model

In [1011]:
class Alternative(TransformerMixin):
    def __init__(self, *models, curr_model=0):
        self.__curr_model = curr_model
        self.models = models

    def fit(self, X, y=None):
        self.get_curr_model().fit(X, y)
        return self

    def predict(self, X):
        return self.get_curr_model().predict(X)

    def transform(self, X):
        res = self.get_curr_model().transform(X)
        return res

    def set_curr_model(self, model_index: int):
        self.__curr_model = model_index

    def get_curr_model(self):
        return self.models[self.__curr_model]

    def set_params(self, **kwargs):
        if 'curr_model' in kwargs:
            self.set_curr_model(kwargs['curr_model'])
            del kwargs['curr_model']
        self.get_curr_model().set_params(**kwargs)
        return self

def cartesian_product_of_dict_lists(*multiple_list_of_dicts):
    if len(multiple_list_of_dicts) == 0:
        return []

    if len(multiple_list_of_dicts) == 1:
        return multiple_list_of_dicts[0]

    if len(multiple_list_of_dicts) == 2:
        ret = []
        for dict1 in multiple_list_of_dicts[0]:
            for dict2 in multiple_list_of_dicts[1]:
                ret.append(dict(**dict1, **dict2))
        return ret

    curr = [{}]
    for dct in multiple_list_of_dicts:
        curr = cartesian_product_of_dict_lists(curr, dct)
    return curr

In [1012]:
x_2b_train, x2b_final_test, y_2b_train, y_2b_final_test = train_test_split(x_2b, y_2b, test_size=0.2, random_state=random_state)
del x_2b
del y_2b

x_2b_train, x2b_test, y_2b_train, y_2b_test = (
    train_test_split(x_2b_train, y_2b_train, test_size=0.2, random_state=random_state)
)

In [1013]:
scaler1 = preprocessing.MinMaxScaler()
scaler2 = preprocessing.StandardScaler()
scaler3 = preprocessing.RobustScaler()

scaler = Alternative(
    scaler1, scaler2, scaler3
)

scaler_params = [{
    'scaler__curr_model': [0, 1, 2]
}]
scaler_params_2 = [{
    'scaler': [scaler1, scaler2, scaler3]
}]

In [1014]:
from sklearn.base import TransformerMixin #gives fit_transform method for free

class MyLabelEncoder(TransformerMixin):
    # source: https://stackoverflow.com/a/46619402/7069108
    def __init__(self, columns_to_be_encoded: list[str], *args, **kwargs):
        self.columns_to_be_encoded = columns_to_be_encoded
        self.encoder = preprocessing.LabelEncoder(*args, **kwargs)

    def fit(self, x, y=0):
        for col in self.columns_to_be_encoded:
            self.encoder.fit(x[col])
        return self

    def transform(self, x, y=0):
        x = x.copy()
        for col in self.columns_to_be_encoded:
            x[col] = self.encoder.transform(x[col])
        return x

label_encoder = MyLabelEncoder(["Pos"])
label_encoder_params = [{}]

In [1015]:
imputer1 = SimpleImputer()
imputer2 = KNNImputer()

imputer = Alternative(
    imputer1, imputer2
)

imputer_params = [{
    'imputer__curr_model': [0, 1]
}]
imputer_params_2 = [{
    'imputer': [imputer1, imputer2]
}]

In [1016]:
feature_selector1 = SelectFromModel(estimator=Lasso())
feature_selector2 = SelectKBest()

feature_selector = Alternative(
    feature_selector1,
    feature_selector2
)

feature_selector_params = [{
    'feature_selector__curr_model': [0],
    'feature_selector__estimator__alpha': [0.01, 0.005, 0.001, 0.0005, 0.0001]
}, {
    'feature_selector__curr_model': [1],
    'feature_selector__k': [6, 7, 8, 9],
}]
feature_selector_params_2 = [{
    'feature_selector': [feature_selector1],
    'feature_selector__estimator__alpha': [0.01, 0.005, 0.001, 0.0005, 0.0001]
}, {
    'feature_selector': [feature_selector2],
    'feature_selector__k': [6, 7, 8, 9],
}]

In [1017]:
predictor = LinearRegression()

In [1018]:
pipeline = Pipeline(
    steps=[
        ('encoding', label_encoder),
        ('scaler', scaler),
        ('imputer', imputer),
        ('feature_selector', feature_selector),
        ('predictor', predictor),
    ]
)

In [1019]:
grid_params = cartesian_product_of_dict_lists(
    [{}],
    label_encoder_params,
    scaler_params,
    imputer_params,
    feature_selector_params,
)

In [1020]:
grid_params

[{'scaler__curr_model': [0, 1, 2],
  'imputer__curr_model': [0, 1],
  'feature_selector__curr_model': [0],
  'feature_selector__estimator__alpha': [0.01, 0.005, 0.001, 0.0005, 0.0001]},
 {'scaler__curr_model': [0, 1, 2],
  'imputer__curr_model': [0, 1],
  'feature_selector__curr_model': [1],
  'feature_selector__k': [6, 7, 8, 9]}]

In [1021]:
grid_search_cv = GridSearchCV(
    estimator=pipeline,
    param_grid=grid_params,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
)

grid_search_cv.fit(x_2b_train, y_2b_train.Salaries)

H:\01 Kuliah\01 Dokumen\52 - KASDD\proyek-akhir\venv\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
H:\01 Kuliah\01 Dokumen\52 - KASDD\proyek-akhir\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.139e+16, tolerance: 1.397e+13
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(estimator=Pipeline(steps=[('encoding',
                                        <__main__.MyLabelEncoder object at 0x0000024394AFD2A0>),
                                       ('scaler',
                                        <__main__.Alternative object at 0x0000024392CE94E0>),
                                       ('imputer',
                                        <__main__.Alternative object at 0x0000024394AFFD90>),
                                       ('feature_selector',
                                        <__main__.Alternative object at 0x0000024394AFE9B0>),
                                       ('predictor', LinearRegression())]),
             n_jobs=-1,
             param_grid=[{'feature_selector__curr_model': [0],
                          'feature_selector__estimator__alpha': [0.01, 0.005,
                                                                 0.001, 0.0005,
                                                                 0.0001],
                          'imputer__curr_model': [0, 1],
                          'scaler__curr_model': [0, 1, 2]},
                         {'feature_selector__curr_model': [1],
                          'feature_selector__k': [6, 7, 8, 9],
                          'imputer__curr_model': [0, 1],
                          'scaler__curr_model': [0, 1, 2]}],
             scoring='neg_mean_absolute_error')

### 2B testing

In [1022]:
y_2b_predict = grid_search_cv.predict(x2b_test)
mse = metrics.mean_squared_error(y_2b_test, y_2b_predict)
print(mse**0.5)
print(mse**0.5 / y_2b_test.Salaries.mean())

3470919.0729861655
0.7534994798290079


In [1023]:
abs(y_2b_test.Salaries - y_2b_predict)

1037    1.109691e+06
642     1.544031e+06
3781    7.296422e+05
2266    5.342460e+05
5350    8.691010e+05
            ...     
2544    5.928685e+06
7923    2.188178e+06
2797    5.065138e+05
4144    4.178821e+06
7662    2.317879e+06
Name: Salaries, Length: 1270, dtype: float64

In [1024]:
grid_params_2 = cartesian_product_of_dict_lists(
    label_encoder_params,
    scaler_params_2,
    imputer_params_2,
    feature_selector_params_2,
)

In [1025]:
pipeline2 = Pipeline(
    steps=[
        ('encoding', label_encoder),
        ('scaler', None),
        ('imputer', None),
        ('feature_selector', None),
        ('predictor', predictor),
    ]
)

In [1026]:
grid_search_cv2 = GridSearchCV(
    estimator=pipeline2,
    param_grid=grid_params_2,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
)

grid_search_cv2.fit(x_2b_train, y_2b_train.Salaries)

H:\01 Kuliah\01 Dokumen\52 - KASDD\proyek-akhir\venv\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
H:\01 Kuliah\01 Dokumen\52 - KASDD\proyek-akhir\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.139e+16, tolerance: 1.397e+13
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(estimator=Pipeline(steps=[('encoding',
                                        <__main__.MyLabelEncoder object at 0x0000024394AFD2A0>),
                                       ('scaler', None), ('imputer', None),
                                       ('feature_selector', None),
                                       ('predictor', LinearRegression())]),
             n_jobs=-1,
             param_grid=[{'feature_selector': [SelectFromModel(estimator=Lasso(alpha=0.01))],
                          'feature_selector__estimator__alpha': [0.01, 0.005,
                                                                 0.001, 0.0005,
                                                                 0.0001],
                          'imputer': [SimpleImputer(), KNNImputer()],
                          'scaler': [MinMaxScaler(), StandardScaler(),
                                     RobustScaler()]},
                         {'feature_selector': [SelectKBest()],
                          'feature_selector__k': [6, 7, 8, 9],
                          'imputer': [SimpleImputer(), KNNImputer()],
                          'scaler': [MinMaxScaler(), StandardScaler(),
                                     RobustScaler()]}],
             scoring='neg_mean_absolute_error')

In [1027]:
y_2b_predict2 = grid_search_cv2.predict(x2b_test)
mse2 = metrics.mean_squared_error(y_2b_test, y_2b_predict2)
print(mse2**0.5)
print(mse2**0.5 / y_2b_test.Salaries.mean())

3470919.0729861655
0.7534994798290079
